In [ ]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [ ]:
#imports

import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import string

from string import punctuation
from os import listdir
from collections import Counter

from keras.models import Sequential, save_model
from keras.layers import Dense, Activation

import os
import csv
import pickle

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# with open('xTrainLines.csv', newline='') as f:
#     reader = csv.reader(f)
#     xTrainLines = list(reader)

with open('xTrainLines.txt') as temp_file:
  xTrainLines = [line.rstrip('\n') for line in temp_file]

with open('xTestLines.txt') as temp_file:
  xTestLines = [line.rstrip('\n') for line in temp_file]

with open('yTrain.txt') as temp_file:
  yTrain = [int(line.rstrip('\n')) for line in temp_file]

with open('yTest.txt') as temp_file:
  yTest = [int(line.rstrip('\n')) for line in temp_file]

In [ ]:
with open('vocab.txt') as temp_file:
  vocab = [line.rstrip('\n') for line in temp_file]

In [ ]:
print(vocab)

In [ ]:
# create the tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer()

# fit the tokenizer on the documents
tokenizer.fit_on_texts(vocab)

In [ ]:
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

class Metrics(Callback):
  def on_train_begin(self, logs={}):
    self.val_f1s = []
    self.val_recalls = []
    self.val_precisions = []
 
  def on_epoch_end(self, epoch, logs={}):
    val_predict = (np.asarray(self.model.predict(self.validation_data[0]))).round()
    val_targ = self.validation_data[1]
    _val_f1 = f1_score(val_targ, val_predict)
    _val_recall = recall_score(val_targ, val_predict)
    _val_precision = precision_score(val_targ, val_predict)
    self.val_f1s.append(_val_f1)
    self.val_recalls.append(_val_recall)
    self.val_precisions.append(_val_precision)
    print("— val_f1: %f — val_precision: %f — val_recall %f" %(_val_f1, _val_precision, _val_recall))
  
metrics = Metrics()

In [ ]:
# encode training data set
xTrain = tokenizer.texts_to_matrix(xTrainLines, mode='freq')
print(xTrain.shape)
print(xTrain[0][:20])

xTest = tokenizer.texts_to_matrix(xTestLines, mode='freq')
print(xTest.shape)

In [ ]:
#storing the shape for the future model

n_words = xTest.shape[1]

# define network
model = Sequential()
model.add(Dense(100, input_shape=(n_words,), activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'mse', 'binary_accuracy'])
print(n_words)
print(len(xTrain), len(yTrain))

4580
1567 1567


In [ ]:
# fit network
model.fit(xTrain, yTrain, validation_data=(xTrain, yTrain), epochs=50, verbose=2, callbacks=[metrics])
model.summary()

In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(xTest, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(yTest, y_pred_bool))

In [ ]:
# evaluate
loss, acc, mse, ba = model.evaluate(xTest, yTest, verbose=0)
print('Test Accuracy: %f' % (acc*100))
print('Loss: %f' % (loss*100))
print('MSE: ', mse * 100)

print(type(xTest))
print(type(yTest))
print(type(xTrain))
print(type(yTrain))

Test Accuracy: 64.571428
Loss: 140.693218
MSE:  28.514987230300903
<class 'numpy.ndarray'>
<class 'list'>
<class 'numpy.ndarray'>
<class 'list'>


In [ ]:
#saving the model

#model.save('path_to_model_run')
save_model(model, "path")

In [ ]:
def clean_text(text):
	# split into tokens by white space
	tokens = text.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# filter out stop words
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	# filter out short tokens
	tokens = [word for word in tokens if len(word) > 1]
	#convert to lowercase
	for i in range(len(tokens)):
		tokens[i] = tokens[i].lower()
	
	return tokens

In [ ]:
# classify a text as negative (0) or positivte (1)
def predict_sentiment(review, vocab, tokenizer, model):
	# clean
	tokens = clean_text(review)
	# filter by vocab
	tokens = [w for w in tokens if w in vocab]
	# convert to line
	line = ' '.join(tokens)
	# encode
	encoded = tokenizer.texts_to_matrix([line], mode='freq')
	# prediction
	yhat = model.predict(encoded, verbose=0)
	return round(yhat[0,0])

In [ ]:
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_scoreclass Metrics(Callback):def on_train_begin(self, logs={}):
 self.val_f1s = []
 self.val_recalls = []
 self.val_precisions = []
 
def on_epoch_end(self, epoch, logs={}):
 val_predict = (np.asarray(self.model.predict(self.model.validation_data[0]))).round()
 val_targ = self.model.validation_data[1]
 _val_f1 = f1_score(val_targ, val_predict)
 _val_recall = recall_score(val_targ, val_predict)
 _val_precision = precision_score(val_targ, val_predict)
 self.val_f1s.append(_val_f1)
 self.val_recalls.append(_val_recall)
 self.val_precisions.append(_val_precision)
 print “ — val_f1: %f — val_precision: %f — val_recall %f” %(_val_f1, _val_precision, _val_recall)
 return
 
metrics = Metrics()

In [ ]:
#new_model = tf.keras.models.load_model('path_to_model_run')

In [ ]:
# test positive text
# text1 = "'Xiao Xingchen? Are you alright?' Xue Yang said anxiously as he saw the daozhang walking towards him. 'Yes, I am.' 'Oh, that's good to hear. Don't get hurt. A-Qing will never let me hear the end of it. Stay safe, alright?' 'Of course. I'm sorry that you have to worry about me. 'Oh, it's fine. I just want you to be safe, you know?'"
# print(predict_sentiment(text1, vocab, tokenizer, model))
# # # test negative text
# text2 = "'Xue Yang!' Song Lan's angry voice echoed across the empty village. Xue Yang grinned and shrugged casually. 'What, I didn't hurt him! Why are you so angry?' 'You tricked him! You fooled him! I can't stand you, you criminal!'"
# print(predict_sentiment(text2, vocab, tokenizer, model))

# text3 = "'Hinata!' We're going to be late!' Komaeda called. 'Coming!' Hinata yelled, and he grabbed a slice of toast as he ran out the door. 'Thanks for reminding me,' he said breathlessly before taking a bite of toast. 'No problem,' Komaeda said, laughing. 'It's what friends do, no?' He began skipping down the sidewalk, singing out, 'Last day of school! Last do of school! Aren't you excited, Hinata?' 'Kind of,' his friend replied. 'I just don't know what I'll do this summer. 'We can go to the fairs together! The pool!' 'I never win anything at fairs...' 'Well, with my luck, I'll help you win!' 'Uh, Komaeda, isn't that cheating?' 'But we win, don't we?' 'Yeah, I guess we do...'"
# print(predict_sentiment(text3, vocab, tokenizer, model))

text_and_dropdown = "\"You tricked me.\" Baizan whispered, the realization of everything that occurred finally sinking in, and she collapsed in despair.. \"You lied!\" \"And what about it?\" Chaojun said casually. \"Sure, I'm evil. Sure, I tricked. Sure, I lied. But we're still friends, right?\" \"Never!\" she looked away, tears falling from her eyes. \"I can't believe that this is happening.\" \"Well, you'd better believe it, he snapped back, voice venomous. \"Nobody can stop me. Don't think that you'll succeed whether others failed. You aren't any more special than them. You have a duty to adhere to. Shut up and do your work. I don't want to hear you again.\" \"You!\" \"Shut up!\"" #@param ["'Xiao Xingchen? Are you alright?' Xue Yang said anxiously as he saw the daozhang walking towards him. 'Yes, I am.' 'Oh, that's good to hear. Don't get hurt. A-Qing will never let me hear the end of it. Stay safe, alright?' 'Of course. I'm sorry that you have to worry about me. 'Oh, it's fine. I just want you to be safe, you know?'", "'Xue Yang!' Song Lan's angry voice echoed across the empty village. Xue Yang grinned and shrugged casually. 'What, I didn't hurt him! Why are you so angry?' 'You tricked him! You fooled him! I can't stand you, you criminal!'", "'Hinata!' We're going to be late!' Komaeda called. 'Coming!' Hinata yelled, and he grabbed a slice of toast as he ran out the door. 'Thanks for reminding me,' he said breathlessly before taking a bite of toast. 'No problem,' Komaeda said, laughing. 'It's what friends do, no?' He began skipping down the sidewalk, singing out, 'Last day of school! Last do of school! Aren't you excited, Hinata?' 'Kind of,' his friend replied. 'I just don't know what I'll do this summer. 'We can go to the fairs together! The pool!' 'I never win anything at fairs...' 'Well, with my luck, I'll help you win!' 'Uh, Komaeda, isn't that cheating?' 'But we win, don't we?' 'Yeah, I guess we do...'"] {allow-input: true}
prediction = predict_sentiment(text_and_dropdown, vocab, tokenizer, model)

if (prediction == 0):
    print("Negative.")
    print(prediction)
else:
    print("Positive.")
    print(prediction)


Negative.
0.0


In [ ]:
# saving xTrainLines
rows = zip(xTrainLines)

with open('xTrainLines.csv','w') as result_file:
    wr = csv.writer(result_file)
    for row in rows:
        wr.writerow(row)

# saving xTestLines
rows = zip(xTestLines)

with open('xTestLines.csv','w') as result_file:
    wr = csv.writer(result_file)
    for row in rows:
        wr.writerow(row)

# saving yTrain
rows = zip(yTrain)

with open('yTrain.csv','w') as result_file:
    wr = csv.writer(result_file)
    for row in rows:
        wr.writerow(row)

# saving yTest
rows = zip(yTest)

with open('yTest.csv','w') as result_file:
    wr = csv.writer(result_file)
    for row in rows:
        wr.writerow(row)

In [ ]:
with open('vocabCount.csv', encoding='utf-8-sig', mode='w') as result_file:
  writer=csv.writer(result_file)
  for key, value in vocab.items():
    for (row in rows):
      writer.writerow(tag + ": " + str(count) + '\n')  

In [ ]:
new_model = tf.keras.models.load_model('path_to_model_run5')
xTest2 = xTest.tolist()

print(type(yTest))

loss2, acc2, mse2, ba2 = new_model.evaluate(xTest2, yTest, verbose=0)
print('Loss: %f' % (mse2*100))

<class 'list'>
Loss: 32.621920
